In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
train_df = pd.read_csv('../../final/data/train_set_noisy_2.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,10.490380,-1.0,0.113630,4.738495,-1.000000,99.895590,-1.000000,3.150403,0,0.730432,24.332066,-1.000000,-1.000000,-1.000000,-1.000000,31.471141,-1.0,6.0
1,10.309792,-1.0,2.067524,0.404102,-1.000000,99.015814,208.280364,3.123680,1,1.665866,13.065579,115.010315,60.042365,4.992607,88.383056,30.929377,-1.0,0.0
2,8.374113,-1.0,-1.000000,3.453598,448.296575,103.808207,-1.000000,2.420072,1,1.766657,132.632967,-1.000000,-1.000000,-1.000000,-1.000000,25.122338,-1.0,1.0
3,10.829884,-1.0,2.530899,-1.000000,-1.000000,90.664501,185.317623,3.583503,1,0.345588,31.763527,-1.000000,68.928513,-1.000000,100.232195,32.489653,-1.0,5.0
4,6.650546,-1.0,-1.000000,0.528332,467.058906,101.393019,-1.000000,1.967753,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,19.951638,-1.0,1.0


In [4]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.783006,112.011007,-1.000000,3.308648,348.014906,86.577465,-1.000000,4.082935,0,1.617594,131.868365,-1.000000,-1.000000,-1.000000,-1.000000,35.349018,-1.000000,7
1,12.609666,104.475099,5.445972,-1.000000,-1.000000,75.082198,-1.000000,5.038345,1,0.334722,97.568987,83.161880,-1.000000,7.147270,-1.000000,37.828999,-1.000000,3
2,7.940172,4.925400,3.255130,5.766738,209.031102,75.254082,-1.000000,3.165345,1,1.649478,2.082970,-1.000000,-1.000000,-1.000000,-1.000000,23.820517,-1.000000,4
3,7.785840,103.757096,2.874384,3.839597,-1.000000,81.873581,39.638010,2.852876,1,0.637843,127.412065,38.534760,51.437387,28.150784,73.296289,23.357519,-1.000000,5
4,12.359715,51.274141,0.178269,1.233844,474.589534,76.469181,49.839768,4.848900,1,0.621034,137.433107,55.314614,47.302738,3.808612,60.076211,37.079144,10.501658,4


In [5]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [6]:
train_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [7]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [ ]:
for steps in [int(8e6), int(9e6), int(10e6), int(11e6), int(12e6)]:
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../../final/models/dqn_noisy_2_{steps}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.27     |
|    ep_rew_mean      | -0.76    |
|    exploration_rate | 0.644    |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 959      |
|    time_elapsed     | 312      |
|    total_timesteps  | 300162   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 10.3     |
|    n_updates        | 62540    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.28     |
|    ep_rew_mean      | -0.98    |
|    exploration_rate | 0.203    |
|    success_rate     | 0.1      |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 992      |
|    t

In [ ]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_4_missing_1_7500000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
#utils.diagnose_sample(dqn_model, X_test, y_test, 1)

In [ ]:
# test_df[(test_df.y_pred==1) & (test_df.y_actual==1)]

In [ ]:
test_df.y_pred.value_counts()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_1_11000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_1_11000000.csv', index=False)